In [6]:
import pickle
import sys

from tabulate import tabulate
import numpy as np
from tqdm.auto import tqdm
import os

sys.path.append("../")
sys.path.append("../covid19_inference")

import covid19_soccer

In [7]:
from tabulate import tabulate


In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
countries = ['England', 'Czechia', 'Scotland', 'Spain', 'Italy', 'Slovakia',
       'Germany', 'Austria', 'Belgium', 'France', 'Portugal',
       'Netherlands']

def load(fstr):
    with open(fstr, "rb") as f:
         return pickle.load(f)

traces, models, dls = [], [], []
for country in tqdm(countries):
    #'UEFA-beta=False-country=England-offset_games=0-draw_delay=True-weighted_alpha_prior=0-prior_delay=-1-width_delay_prior=0.1-sigma_incubation=-1.0-median_width_delay=1.0-tune=200-draws=300-max_treedepth=10.pickled'
    model = None
    fstr=lambda tune, draws, max_treedepth: (f"/data.nst/smohr/covid19_soccer_data/main_traces/run"+
        f"-beta=False"+
        f"-country={country}"+
        f"-offset_data=0"+
        f"-prior_delay=-1"+
        f"-median_width_delay=1.0"+
        f"-interval_cps=10.0"+
        f"-f_fem=0.33"+
        f"-len=normal"+
        f"-abs_sine=False"+
        f"-t={tune}"+
        f"-d={draws}"+
        f"-max_treedepth={max_treedepth}.pkl")
    #print(fstr(4000, 8000, 12))
    if os.path.exists(fstr(4000, 8000, 12)):
        try:
            model, trace = load(fstr(4000, 8000, 12))
            print(f"Use 8000 sample runs for {country}")
        except Exception as exc:
            print(exc)
            pass
    if model is None and os.path.exists(fstr(2000, 4000, 12)):
        try:
            model, trace = load(fstr(2000, 4000, 12))
            print(f"Use 4000 sample runs for {country}")
        except Exception as exc:
            print(exc)
            pass
    if model is None and os.path.exists(fstr(1000, 1500, 12)):
        try: 
            model, trace = load(fstr(1000, 1500, 12))
            print(f"Use 1500 sample runs for {country}")
        except Exception as exc:
            print(exc)
            pass
    if model is None: 
        print(fstr(1000, 1500, 12), " not found")
        continue
    
    # Remove chains with likelihood larger than -200, should only be the case for 2 chains in France
    mask = (np.mean(trace.sample_stats.lp, axis=1)>-200)
    trace.posterior = trace.posterior.sel(chain=~mask)
    
    dl = covid19_soccer.dataloader.Dataloader_gender(countries=[country])
    models.append(model)
    traces.append(trace)
    dls.append(dl)

  0%|          | 0/12 [00:00<?, ?it/s]

Use 4000 sample runs for England


INFO     [covid19_inference.data_retrieval.retrieval] Successfully loaded OxCGRT.csv.gz from /tmp/covid19_data/, skipping download.
  8%|▊         | 1/12 [00:09<01:49,  9.95s/it]

Use 4000 sample runs for Czechia


INFO     [covid19_inference.data_retrieval.retrieval] Successfully loaded OxCGRT.csv.gz from /tmp/covid19_data/, skipping download.
 17%|█▋        | 2/12 [00:15<01:16,  7.61s/it]

...

INFO     [covid19_inference.data_retrieval.retrieval] Successfully loaded OxCGRT.csv.gz from /tmp/covid19_data/, skipping download.
 83%|████████▎ | 10/12 [01:05<00:12,  6.39s/it]

Use 4000 sample runs for Portugal


INFO     [covid19_inference.data_retrieval.retrieval] Successfully loaded OxCGRT.csv.gz from /tmp/covid19_data/, skipping download.
 92%|█████████▏| 11/12 [01:12<00:06,  6.70s/it]

Use 4000 sample runs for Netherlands


INFO     [covid19_inference.data_retrieval.retrieval] Successfully loaded OxCGRT.csv.gz from /tmp/covid19_data/, skipping download.
100%|██████████| 12/12 [01:19<00:00,  6.60s/it]


In [10]:
from multiprocessing import Pool
import arviz as az

def get_max_rhat(country, trace):
    rhat = az.rhat(trace)
    chosen_vars = []
    for var in list(rhat.data_vars):
        if "lambda" in var:
            chosen_vars.append(var)
        if "R_t" in var:
            chosen_vars.append(var)
        if "factor_female" in var:
            chosen_vars.append(var)
        if "alpha" in var:
            chosen_vars.append(var)

    rhats_R_t = []
    rhat_max = rhat.max()
    for var in chosen_vars:
        rhats_R_t.append(float(rhat_max[var]))
    rhats_all = []
    for var in rhat.data_vars:
        rhats_all.append(float(rhat_max[var]))
    print(f"{country} max rhat of R_t {max(rhats_R_t)} ")
    print(f"{country} max overall rhat {max(rhats_all)} ")
        
    return rhats_R_t, rhats_all
    

#def get_max_rhat(country, trace):
#    return 3.
    
rhats = []
    
with Pool(processes=6) as pool:
    results = []
    for i, country in enumerate(countries):
        trace = traces[i]    
        result = pool.apply_async(get_max_rhat, (country, trace)) 
        results.append(result)
    for res in results:
        rhats.append(res.get())


/data.nst/smohr/anaconda3/envs/pymc3_new/lib/python3.8/site-packages/arviz/stats/diagnostics.py:561: RuntimeWarning: invalid value encountered in double_scalars
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)
/data.nst/smohr/anaconda3/envs/pymc3_new/lib/python3.8/site-packages/arviz/stats/diagnostics.py:561: RuntimeWarning: invalid value encountered in double_scalars
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)
/data.nst/smohr/anaconda3/envs/pymc3_new/lib/python3.8/site-packages/arviz/stats/diagnostics.py:561: RuntimeWarning: invalid value encountered in double_scalars
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)


England max rhat of R_t 1.332204947696311 
England max overall rhat 1.8791322077014203 


/data.nst/smohr/anaconda3/envs/pymc3_new/lib/python3.8/site-packages/arviz/stats/diagnostics.py:561: RuntimeWarning: invalid value encountered in double_scalars
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)


Czechia max rhat of R_t 1.0012462581859247 
Czechia max overall rhat 1.1594230125924612 


/data.nst/smohr/anaconda3/envs/pymc3_new/lib/python3.8/site-packages/arviz/stats/diagnostics.py:561: RuntimeWarning: invalid value encountered in double_scalars
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)
/data.nst/smohr/anaconda3/envs/pymc3_new/lib/python3.8/site-packages/arviz/stats/diagnostics.py:561: RuntimeWarning: invalid value encountered in double_scalars
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)


...

Germany max rhat of R_t 1.0079659606863034 
Germany max overall rhat 1.415311079294106 


/data.nst/smohr/anaconda3/envs/pymc3_new/lib/python3.8/site-packages/arviz/stats/diagnostics.py:561: RuntimeWarning: invalid value encountered in double_scalars
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)


Austria max rhat of R_t 1.0043118799366977 

Austria max overall rhat 1.1471551517874619 Belgium max rhat of R_t 1.0070083619523587 
Belgium max overall rhat 1.2181615199135256 


/data.nst/smohr/anaconda3/envs/pymc3_new/lib/python3.8/site-packages/arviz/stats/diagnostics.py:561: RuntimeWarning: invalid value encountered in double_scalars
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)


France max rhat of R_t 1.0083534362195294 
France max overall rhat 1.8218383321435438 
Portugal max rhat of R_t 1.001019971412043 
Portugal max overall rhat 1.1384544260989433 
Netherlands max rhat of R_t 1.0269737256974478 
Netherlands max overall rhat 1.8255915904861921 


In [11]:
table = []
for i, country in enumerate(countries):
    table.append([country, f"{max(rhats[i][0]):.2f}", f"{max(rhats[i][1]):.2f}"])
header = ["Country", "Max. R-hat of relevant variables", "Max. R-hat of all variables"]

print(tabulate(table, header, tablefmt="latex_raw"))

\begin{tabular}{lrr}
\hline
 Country     &   Max. R-hat of relevant variables &   Max. R-hat of all variables \\
\hline
 England     &                               1.33 &                          1.88 \\
 Czechia     &                               1    &                          1.16 \\
 Scotland    &                               1.01 &                          1.1  \\
 Spain       &                               1.05 &                          2.24 \\
 Italy       &                               1.01 &                          1.1  \\
 Slovakia    &                               1    &                          1.15 \\
 Germany     &                               1.01 &                          1.42 \\
 Austria     &                               1    &                          1.15 \\
 Belgium     &                               1.01 &                          1.22 \\
 France      &                               1.01 &                          1.82 \\
 Portugal    &                